In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("data/hius.csv")
df_2 = pd.read_csv("data/quickcharts_hius.csv")

#the first row is the description of the column so we should drop that as well
df = df.drop(0, axis=0)
df.head()

In [ ]:
df_2 = df_2.drop(0, axis=0)
df_2.head()

In [ ]:
# the difference between the two datasets
list(set(df.columns) - set(df_2.columns))

In [ ]:
for column in df.columns:
    print("{} has values:\n{}\n".format(column, df[column].unique()))
    print("-"*100)

In [ ]:
#figuring out how many nans per column
print(df.shape)
df.isna().sum()

In [ ]:
#separating diseases

disList = df['GHO (CODE)'].unique()

diseases = []
for i in disList:
    tempDis = df[df['GHO (CODE)'] == i]
    diseases.append(tempDis)
    
#nan values for each disease df
for i in range(len(diseases)):
    print(diseases[i].head())

# for i in diseases[1]:
    
#     x = i.values[:, :-1]

#     nan_count = i.isna().sum()
#     nan_percentage = [count / x.shape[0] for count in nan_count]
#     columns = i.columns

#     threshold = 0.6

#     # And let's plot those
#     fig1 = plt.figure(figsize=(30,15))
#     ax = fig1.add_subplot(111)
#     plt.axhline(y=threshold,linewidth=1, color="k")
#     ax.bar(columns, nan_percentage)
#     plt.xticks(rotation=90)
#     ax.set_xlabel('Column')
#     ax.set_ylabel('NaN Count')
#     ax.set_title('Columns with missing values')
    

### Conclusion:

The set is as good as complete, but some columns only apply to some diseases/conditions
So, we will remove all unimportant columns for each disease.

In [ ]:
for k in range(len(diseases)):
    i = diseases[k]
    x = i.values[:, :-1]

    nan_count = i.isna().sum()
    nan_percentage = [count / x.shape[0] for count in nan_count]
    columns = i.columns

    threshold = 0.6

    too_much_cols = []
    for j in range(len(columns)):
        if nan_percentage[j] >= threshold:
            too_much_cols.append(columns[j])
    diseases[k].drop(too_much_cols, axis=1, inplace=True)
    
    
diseases[0].head()

In [ ]:
print(diseases[0].dtypes)

All values are objects, change to numeric values.

In [ ]:
print(len(diseases))

In [ ]:
print(diseases[10].filter(items=['YEAR (CODE)', 'Display Value']))
for i in range(len(diseases)):
    for index, row in diseases[i].iterrows():
        if type(row['YEAR (CODE)']) == str:
            split_years = row['YEAR (CODE)'].split('-')
            if len(split_years) > 1:
                diseases[i].loc[index, 'YEAR (CODE)'] = int((int(split_years[0]) + int(split_years[1]))/2)  
        if diseases[i].loc[index, 'Display Value'] == 'No' or diseases[i].loc[index, 'Display Value'] == 'No data' or diseases[i].loc[index, 'Display Value'] == '' or diseases[i].loc[index, 'Display Value'] == ' ':
            diseases[i].loc[index, 'Display Value'] = np.nan
        if type(diseases[i].loc[index, 'Display Value']) == str:
            while " " in diseases[i].loc[index, 'Display Value']:
                split = row['Display Value'].split(" ")
                diseases[i].loc[index, 'Display Value'] = split[0]
            while "[" in diseases[i].loc[index, 'Display Value']:
                split = row['Display Value'].split("[")
                diseases[i].loc[index, 'Display Value'] = split[0]
            if "<" in diseases[i].loc[index, 'Display Value']:
                split = row['Display Value'].split("<")
                diseases[i].loc[index, 'Display Value'] = split[1]
        
        print('disease ', i, ' ', diseases[i].loc[index, 'YEAR (CODE)'], ' ', diseases[i].loc[index, 'Display Value'])

In [ ]:
diseases[53].loc[diseases[53]['YEAR (CODE)'] == '2009', 'Display Value'] = np.nan
diseases[95].loc[diseases[95]['YEAR (CODE)'] == '2013', 'Display Value'] = 53960

In [ ]:
# *

In [ ]:
print(diseases[0].iloc[0]['GHO (DISPLAY)'])

from statsmodels.tsa.seasonal import seasonal_decompose


for i in range (0,len(diseases)):
    diseases[i].rename(columns={"YEAR (CODE)": "period", "Display Value": "value"}, inplace=True)    
    print(diseases[i].columns)
    decompfreq = 12  # 12 months seasonality

    decomposition = seasonal_decompose(
        diseases[i].set_index("period").value.interpolate("linear"),
        freq=decompfreq,
        model=model)

    trend = decomposition.trend


    fig, ax = plt.subplots(figsize=(18,6))
    diseases[i].plot(x="period", y="value", ax=ax, label="observed", c='lightgrey', title=diseases[i].iloc[0]['GHO (DISPLAY)'])
    trend.plot(ax=ax, label="trend")
    plt.legend(loc='upper left')
